In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers
# !pip install mxnet
!pip install gluonnlp==0.8.0
!pip install tqdm pandas
# !pip install sentencepiece
#!pip install torch

In [ ]:
import math
import os
from tempfile import TemporaryDirectory
from typing import Tuple
import numpy as np
import logging
import json
import pandas as pd

import torch
from torch import nn,Tensor
import torch.nn.functional as F
from torch.nn import Transformer, TransformerEncoder, TransformerEncoderLayer
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from transformers.optimization import AdamW, get_cosine_schedule_with_warmup

from tqdm import tqdm

# for koBART
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration

import re

In [ ]:
# train dataset
#train = pd.read_csv('train_lyrics.csv', names=['lyrics', 'syllables'])
train = pd.read_csv(
    "final_syl.csv", sep="\t", header=None, names=["lyrics"], encoding="utf-8-sig"
)
train = train[:len(train) // 2]

# 데이터프레임을 출력하여 확인합니다.
train.head()

,lyrics
0,날_ _이 [sep] 더_ _워_ _지_ _고 [sep] 뻐_ _근_ _해_ _진 ...
1,달_ _래_ _러 [sep] 우_ _리 [sep] 놀_ _러 [sep] 갈_ _까_...
2,그_ _대_ _도 [sep] 나_ _와 [sep] 같_ _은 [sep] 생_ _각_...
3,난 [sep] 믿_ _어_ _요 [sep] 그_ _대_ _는 [sep] 어_ _떤_...
4,생_ _각_ _해_ _보_ _니 [sep] 너_ _무_ _도 [sep] 바_ _쁘_...


In [ ]:
x = train.iloc[10000, 0]
print(re.sub('_ _', '', x))

당신이 [sep] 맞다면 [sep]


In [ ]:
x = train.iloc[1, 0]
print("<s>" + re.sub('\[sep\]', '', re.sub(' \[sep\] ', ' ', x)) + "</s>")

<s>나는 나는 갯바위 당신은 나를 사랑하는 파도 어느 고운 바람 불던날 잔잔히 다가와 부드러운 손길로 나를 감싸고 향기로운 입술도 내게 주었지 세찬 비바람에 내몸이 패이고 이는 파도에 깨끗이 부서져도 나의 생은 당신의 조각품인것을 나는 당신으로 인해 아름다운 것을 나는 나는 갯바위 당신은 나를 사랑하는 파도 우린 오늘도 마주보며 이렇게 서있네 세찬 비바람에 내몸이 패이고 이는 파도에 깨끗이 부서져도 나의 생은 당신의 조각품인것을 나는 당신으로 인해 아름다운 것을 나는 나는 갯바위 당신은 나를 사랑하는 파도 우린 오늘도 마주보며 이렇게 서있네 나는 나는 갯바위 당신은 나를 사랑하는 파도 우린 오늘도 마주보며 이렇게 서있네 이렇게 서있네 </s>


In [ ]:
x = train.iloc[1, 1]
print("<s>" + re.sub('([0-9])', '<unused\\1>', re.sub(' ', '', re.sub('\[sep\]', '', x))) + "</s>")

<s><unused2><unused2><unused3><unused3><unused2><unused4><unused2><unused2><unused2><unused2><unused3><unused3><unused3><unused4><unused3><unused2><unused3><unused4><unused3><unused2><unused3><unused2><unused4><unused3><unused3><unused2><unused3><unused3><unused4><unused2><unused2><unused3><unused6><unused2><unused4><unused2><unused4><unused2><unused2><unused2><unused3><unused3><unused2><unused4><unused2><unused2><unused3><unused4><unused3><unused3><unused2><unused4><unused3><unused3><unused2><unused3><unused3><unused4><unused2><unused2><unused3><unused6><unused2><unused4><unused2><unused4><unused2><unused2><unused2><unused3><unused3><unused2><unused4><unused2><unused2><unused3><unused4><unused3><unused3><unused2><unused2><unused3><unused3><unused2><unused4><unused2><unused2><unused3><unused4><unused3><unused3><unused3><unused3></s>


In [ ]:
re.sub(' \[sep\] ', '<unused0>', "난 [sep] 오늘도 [sep] 조용히 [sep] 그댈 [sep] 그리워하네")

'난<unused0>오늘도<unused0>조용히<unused0>그댈<unused0>그리워하네'

In [ ]:
def make_decoder_input(x, eos_token=1, pad_token=3):
  # Use torch.roll to shi
  # shift the elements to the right by one place
  x = torch.roll(x, shifts=1, dims=1)

  # Set the first column to </s>
  x[:, 0] = eos_token

  # Check where the value is '3'
  matches = (x == pad_token)

  # Use torch.where to get the indices
  indices = [torch.where(m)[0][0] if m.sum() > 0 else 0 for m in matches]

  for i, index in enumerate(indices):
    x[i, index - 1] = pad_token

  return x

In [ ]:
# import gc
# gc.collect()
# del model

In [ ]:
y = train['lyrics'].apply(lambda x: re.sub('_ _', '', x))
type(y[:10])

pandas.core.series.Series

In [ ]:
class MyDataset(Dataset):
    def __init__(self, file, tokenizer, max_length=256):
        self.inputs = []
        lyrics_list = file['lyrics'].apply(lambda x: re.sub('_ _', '', x))
        for lyric in lyrics_list:
          notes = [f"<unused{len(word)}>" if word != "[sep]" else '<unused0>' for word in lyric.split()]
          self.inputs.append("<s>" + ' '.join(notes) + "</s>") # 더 나은 코드가 있을텐데..
        self.outputs = list(lyrics_list.apply(lambda x: "<s>" + re.sub('\[sep\]', '<unused0>', x) + "</s>"))
        # self.inputs = list(file['syllables'].apply(lambda x: "<s>" + re.sub('([0-9])', '<unused\\1>', re.sub(' ', '', re.sub('\[sep\]', '', x))) + "</s>")
        # self.outputs = list(file['lyrics'].apply(lambda x: "<s>" + re.sub('\[sep\]', '', re.sub(' \[sep\] ', ' ', x)) + "</s>")
        self.tokenizer = tokenizer
        # self.device = 'cuda' if torch.cuda.is_available() else 'cpu'

        print(self.inputs[0])
        print(self.outputs[0])

        input_tokens = self.tokenizer(self.inputs, padding="max_length", max_length=max_length, truncation=True, return_tensors='pt')
        output_tokens = self.tokenizer(self.outputs, padding="max_length", max_length=max_length, truncation=True, return_tensors='pt')

        self.input_ids = input_tokens['input_ids']
        self.labels = output_tokens['input_ids']
        self.decoder_input_ids = make_decoder_input(self.labels)

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.decoder_input_ids[idx], self.labels[idx]

In [ ]:
class KoBARTConditionalGeneration(nn.Module):
    def __init__(self, device = 'cuda' if torch.cuda.is_available() else 'cpu'):
        super().__init__()
        self.model = BartForConditionalGeneration.from_pretrained('gogamza/kobart-base-v1').to(device)
        self.model.train()
        self.bos_token = '<s>'
        self.eos_token = '</s>'
        self.sep_token = '<unused0>'
        self.device = device

        self.tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v1')
        self.pad_token_id = self.tokenizer.pad_token_id

    def forward(self, input_ids, decoder_input_ids, labels):
        attention_mask = input_ids.ne(self.pad_token_id).float().to(self.device) #input_ids.ne()안을 올바르게 채워주세요
        decoder_attention_mask = decoder_input_ids.ne(self.pad_token_id).float().to(self.device) #decoder_input_ids.ne()안을 올바르게 채워주세요

        return self.model(input_ids=input_ids.to(self.device),
                          attention_mask=attention_mask,
                          decoder_input_ids=decoder_input_ids.to(self.device),
                          decoder_attention_mask=decoder_attention_mask,
                          labels=labels, return_dict=True)

    def generate(self, *args, **kwargs):
      return self.model.generate(*args, **kwargs)

In [ ]:
def dataset_split(dataset, ratio):
    train_size = int(ratio * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])
    return train_dataset, val_dataset

In [ ]:
model = KoBARTConditionalGeneration()

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [ ]:
batch_size = 32 # 자신의 현재 GPU 성능에 맞게 조절해 주세요.

dataset = MyDataset(train, model.tokenizer)
train_dataset, val_dataset = dataset_split(dataset, 0.8)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

<s><unused2> <unused0> <unused4> <unused0> <unused4> <unused0> <unused2> <unused0></s>
<s>날이 <unused0> 더워지고 <unused0> 뻐근해진 <unused0> 몸을 <unused0></s>


In [ ]:
logger = logging.getLogger()  ###진행 과정에서 로깅 포인트가 발생할 경우 로깅을 하기 위한 코드입니다. 수정하실 필요 없습니다
logger.setLevel(logging.INFO)  ###진행 과정에서 로깅 포인트가 발생할 경우 로깅을 하기 위한 코드입니다. 수정하실 필요 없습니다

# Config
epochs = 10
warmup_ratio = 0.1
learning_rate = 3e-5
grad_clip = 1.0
# train_log_interval = 100 ##train이 100번 이루어질 때마다 logging되도록 한 코드입니다
# validation_interval = 1000 ##1000번의 train이 끝날 때마다 validation을 수행
save_interval = 1000 ##save point는 1000번의 train

# optimizer
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(
        nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(
        nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate, correct_bias=False)

# scheduler
data_len = len(train_dataloader)
num_train_steps = int(data_len / batch_size * epochs)
#num_warmup_steps = int(num_train_steps * warmup_ratio)
#scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_train_steps)

In [ ]:
# logging data info : logging 정보를 지정된 로깅 포인트마다 제시해 주도록 하는 코드입니다.
logging.info(f'data length {data_len}')
logging.info(f'num_train_steps : {num_train_steps}')
#logging.info(f'num_warmup_steps : {num_warmup_steps}')


#하드웨어 가속기에 따라 device 값을 설정해주는 코드입니다. 따로 수정하실 필요 없습니다!
device = 'cuda' if torch.cuda.is_available() else 'cpu'


def _validate( #학습 시 검증을 위한 것입니다.
        model: KoBARTConditionalGeneration,
        dev_dataloader: DataLoader,
        device: torch.device,
        logger: logging.Logger,
        global_step: int,
):
    model.eval()  # 모델을 평가 모드로 전환합니다.
    loss_list = []
    for batch_data in tqdm(dev_dataloader, desc="[EVAL]"):
        with torch.no_grad():
            input_ids, decoder_input_ids, labels = batch_data
            model_outputs = model.forward(input_ids, decoder_input_ids, labels)
            loss_list.append(model_outputs.loss.item())

    mean_loss = np.mean(loss_list)
    logger.info(f"[EVAL] global_step:{global_step} loss:{mean_loss:.4f} perplexity:{math.exp(mean_loss):.4f}")
    model.train()

    return mean_loss

def bart_train(model, optimizer, train_dataloader, val_dataloader, device):
    model.to(device)  # 모델 학습을 설정된 device (CPU, cuda) 위에서 진행하도록 하는 코드입니다.
    model.train() # 모델을 학습 모드로 전환합니다.
    loss_list_between_log_interval = []
    for epoch_id in range(epochs):
        for step_index, batch_data in tqdm(enumerate(train_dataloader), f"[TRAIN] Epoch:{epoch_id+1}", total=len(train_dataloader)):
                global_step = len(train_dataloader) * epoch_id + step_index + 1
                optimizer.zero_grad()

                input_ids, decoder_input_ids, labels = batch_data
                #print(input_ids)

                model_outputs = model(input_ids, decoder_input_ids, labels)

                loss = model_outputs.loss
                loss.backward()

                torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
                optimizer.step()
                #scheduler.step()

                # for logging
                loss_list_between_log_interval.append(model_outputs.loss.item())

                #if global_step % train_log_interval == 0:
        mean_loss = np.mean(loss_list_between_log_interval)
        logger.info(
            f"EP:{epoch_id} global_step:{global_step} "
            f"loss:{mean_loss:.4f} perplexity:{math.exp(mean_loss):.4f}"
        )
        loss_list_between_log_interval.clear()

                # if global_step % validation_interval == 0:
        dev_loss = _validate(model, val_dataloader, device, logger, global_step)

        #print("saving model...")
        state_dict = model.state_dict()
        model_path = os.path.join('/content/drive/MyDrive/ML/my_AIKU/AIKU_project/Kim_yina/kobart', f"epoch_{epoch_id}.pth")
        logger.info(f"global_step: {global_step} model saved at {model_path}")
        torch.save(state_dict, model_path)

    return model

INFO:root:data length 18063
INFO:root:num_train_steps : 5644


In [ ]:
model = bart_train(model, optimizer, train_dataloader, val_dataloader, device)

In [ ]:
model = KoBARTConditionalGeneration()
model.load_state_dict(torch.load('/content/drive/MyDrive/ML/my_AIKU/AIKU_project/Kim_yina/kobart/epoch_3.pth'))
model.eval()

In [ ]:
def syllables2lyrics(model, notes):
  notes = [f"<unused{n}> <unused0>" for n in notes]
  input_text = "<s> + " " ".join(notes) + </s>
  input_ids = model.tokenizer.encode(input_text, add_special_tokens=True, return_tensors="pt").to('cuda')
  generated_ids = model.generate(input_ids, max_length=50, num_beams=4)
  generated_text = model.tokenizer.decode(generated_ids[0], skip_special_tokens=True)
  print("Generated Text:", generated_text)